# Bagging

Let's implement some bagged LGBM models

In [1]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pickle
from pathlib import Path

### USE FOR LOCAL JUPYTER NOTEBOOKS ###
DOWNLOAD_DIR = Path('../download')
DATA_DIR = Path('../data')
SUBMISSIONS_DIR = Path('../submissions')
MODEL_DIR = Path('../models')
#######################################

X = pd.read_csv(DOWNLOAD_DIR / 'train_values.csv', index_col='building_id')
categorical_columns = X.select_dtypes(include='object').columns
bool_columns = [col for col in X.columns if col.startswith('has')]

X_test = pd.read_csv(DOWNLOAD_DIR / 'test_values.csv', index_col='building_id')
y = pd.read_csv(DOWNLOAD_DIR / 'train_labels.csv', index_col='building_id')

In [2]:
import wandb
wandb.login()

wandb: Currently logged in as: theadammurphy (use `wandb login --relogin` to force relogin)


True

In [3]:
wandb.init(project='earthquake_damage_competition')

wandb: wandb version 0.10.21 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [3]:
X_test.shape

(86868, 38)

In [4]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer

label_enc = LabelEncoder()

t = [('ord_encoder', OrdinalEncoder(dtype=int), categorical_columns)]
ct = ColumnTransformer(transformers=t, remainder='passthrough')

In [7]:
X_all_ints = ct.fit_transform(X)
y = label_enc.fit_transform(np.ravel(y))

In [8]:
# Note that append for pandas objects works differently to append with
# python objects e.g. python append modifes the list in-place
# pandas append returns a new object, leaving the original unmodified
not_categorical_columns = X.select_dtypes(exclude='object').columns
cols_ordered_after_ordinal_encoding = categorical_columns.append(not_categorical_columns)

In [9]:
geo_cols = pd.Index(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id'])
cat_cols_plus_geo = categorical_columns.append(geo_cols)

In [10]:
train_data = lgb.Dataset(X_all_ints,
                        label=y,
                        feature_name=list(cols_ordered_after_ordinal_encoding),
                        categorical_feature=list(cat_cols_plus_geo))

In [11]:
# Taken from the docs for lgb.train and lgb.cv
# Helpful Stackoverflow answer: 
# https://stackoverflow.com/questions/50931168/f1-score-metric-in-lightgbm
from sklearn.metrics import f1_score

def get_ith_pred(preds, i, num_data, num_class):
    """
    preds: 1D NumPY array
        A 1D numpy array containing predicted probabilities. Has shape
        (num_data * num_class,). So, For binary classification with 
        100 rows of data in your training set, preds is shape (200,), 
        i.e. (100 * 2,).
    i: int
        The row/sample in your training data you wish to calculate
        the prediction for.
    num_data: int
        The number of rows/samples in your training data
    num_class: int
        The number of classes in your classification task.
        Must be greater than 2.
    
    
    LightGBM docs tell us that to get the probability of class 0 for 
    the 5th row of the dataset we do preds[0 * num_data + 5].
    For class 1 prediction of 7th row, do preds[1 * num_data + 7].
    
    sklearn's f1_score(y_true, y_pred) expects y_pred to be of the form
    [0, 1, 1, 1, 1, 0...] and not probabilities.
    
    This function translates preds into the form sklearn's f1_score 
    understands.
    """
    # Does not work for binary classification, preds has a different form
    # in that case
    assert num_classs > 2
    
    preds_for_ith_row = [preds[class_label * num_data + i]
                        for class_label in range(num_class)]
    
    # The element with the highest probability is predicted
    return np.argmax(preds_for_ith_row)
    
def lgb_f1_micro(preds, train_data):
    y_true = train_data.get_label()
    
    num_data = len(y_true)
    num_class = 3
    
    y_pred = []
    for i in range(num_data):
        ith_pred = get_ith_pred(preds, i, num_data, num_class)
        y_pred.append(ith_pred)
    
    return 'f1', f1_score(y_true, y_pred, average='micro'), True

In [32]:
param = {'num_leaves': 120,
         'min_child_samples': 40,
         'learning_rate': 0.2,
         'boosting_type': 'goss',
         'objective': 'multiclass',
         'num_class': 3,
         'verbosity': -1,
         'num_threads': 8,
         'seed': 1}

bags = 50
bagged_prediction = np.zeros(X_test.shape[0])

for n in range(bags):
    print(f'Bag: {n}')
    param['seed'] += n
    booster = lgb.train(param,
                        train_data,
                        30,
                        categorical_feature=list(cat_cols_plus_geo),
                        feval=lgb_f1_micro)
    
    prob_preds = booster.predict(ct.transform(X_test))
    preds = [np.argmax(p) + 1 for p in prob_preds]
    bagged_prediction += preds

Bag: 0
Bag: 1
Bag: 2
Bag: 3
Bag: 4
Bag: 5
Bag: 6
Bag: 7
Bag: 8
Bag: 9


How am I supposed to know if this model is better without testing it beforehand? Let's just make a model and submit it and see what the score is first and then worry about more submissions after. If I have a range of submissions I want to try, I can just build them up and submit new ones each day. I can even create a list of those I want to submit

In [33]:
np.unique(bagged_prediction / 10)

array([1. , 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2,
       2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3. ])

In [34]:
bagged_prediction /= 10

In [35]:
rounded_preds = [int(round(pred)) for pred in bagged_prediction]

In [36]:
sub_format = pd.read_csv(DOWNLOAD_DIR / 'submission_format.csv',
                         index_col='building_id')

my_sub = pd.DataFrame(data=rounded_preds,
                      columns=sub_format.columns,
                      index=sub_format.index)

title = '03-09 - LGBM API - lgbm_02_02 hyperparams - 10 bags, 30 rounds'

my_sub.to_csv(SUBMISSIONS_DIR / f'{title}.csv')

Woo jumped to 0.7469 
From 228 to 167

In [37]:
def build_and_submit_bagged_lgbm(num_bags, title):
    param = {'num_leaves': 120,
         'min_child_samples': 40,
         'learning_rate': 0.2,
         'boosting_type': 'goss',
         'objective': 'multiclass',
         'num_class': 3,
         'verbosity': -1,
         'num_threads': 8,
         'seed': 1}

    bagged_prediction = np.zeros(X_test.shape[0])

    for n in range(num_bags):
        print(f'Bag: {n}')
        param['seed'] += n
        booster = lgb.train(param,
                            train_data,
                            30,
                            categorical_feature=list(cat_cols_plus_geo),
                            feval=lgb_f1_micro)

        prob_preds = booster.predict(ct.transform(X_test))
        preds = [np.argmax(p) + 1 for p in prob_preds]
        bagged_prediction += preds
        
    bagged_prediction /= num_bags
    rounded_preds = [int(round(pred)) for pred in bagged_prediction]
    
    sub_format = pd.read_csv(DOWNLOAD_DIR / 'submission_format.csv',
                         index_col='building_id')

    my_sub = pd.DataFrame(data=rounded_preds,
                          columns=sub_format.columns,
                          index=sub_format.index)

    my_sub.to_csv(SUBMISSIONS_DIR / f'{title}.csv')
    print('Submission created successfully!')

In [38]:
title = '03-09 - LGBM API - lgbm_02_02 hyperparams - 50 bags, 30 rounds'
build_and_submit_bagged_lgbm(50, title)

Bag: 0
Bag: 1
Bag: 2
Bag: 3
Bag: 4
Bag: 5
Bag: 6
Bag: 7
Bag: 8
Bag: 9
Bag: 10
Bag: 11
Bag: 12
Bag: 13
Bag: 14
Bag: 15
Bag: 16
Bag: 17
Bag: 18
Bag: 19
Bag: 20
Bag: 21
Bag: 22
Bag: 23
Bag: 24
Bag: 25
Bag: 26
Bag: 27
Bag: 28
Bag: 29
Bag: 30
Bag: 31
Bag: 32
Bag: 33
Bag: 34
Bag: 35
Bag: 36
Bag: 37
Bag: 38
Bag: 39
Bag: 40
Bag: 41
Bag: 42
Bag: 43
Bag: 44
Bag: 45
Bag: 46
Bag: 47
Bag: 48
Bag: 49
Submission created successfully!


This got 0.7478!! From 168 to 132 (36 place jumps)

In [40]:
title = '03-09 - LGBM API - lgbm_02_02 hyperparams - 250 bags, 30 rounds'
build_and_submit_bagged_lgbm(250, title)

Bag: 0
Bag: 1
Bag: 2
Bag: 3
Bag: 4
Bag: 5
Bag: 6
Bag: 7
Bag: 8
Bag: 9
Bag: 10
Bag: 11
Bag: 12
Bag: 13
Bag: 14
Bag: 15
Bag: 16
Bag: 17
Bag: 18
Bag: 19
Bag: 20
Bag: 21
Bag: 22
Bag: 23
Bag: 24
Bag: 25
Bag: 26
Bag: 27
Bag: 28
Bag: 29
Bag: 30
Bag: 31
Bag: 32
Bag: 33
Bag: 34
Bag: 35
Bag: 36
Bag: 37
Bag: 38
Bag: 39
Bag: 40
Bag: 41
Bag: 42
Bag: 43
Bag: 44
Bag: 45
Bag: 46
Bag: 47
Bag: 48
Bag: 49
Bag: 50
Bag: 51
Bag: 52
Bag: 53
Bag: 54
Bag: 55
Bag: 56
Bag: 57
Bag: 58
Bag: 59
Bag: 60
Bag: 61
Bag: 62
Bag: 63
Bag: 64
Bag: 65
Bag: 66
Bag: 67
Bag: 68
Bag: 69
Bag: 70
Bag: 71
Bag: 72
Bag: 73
Bag: 74
Bag: 75
Bag: 76
Bag: 77
Bag: 78
Bag: 79
Bag: 80
Bag: 81
Bag: 82
Bag: 83
Bag: 84
Bag: 85
Bag: 86
Bag: 87
Bag: 88
Bag: 89
Bag: 90
Bag: 91
Bag: 92
Bag: 93
Bag: 94
Bag: 95
Bag: 96
Bag: 97
Bag: 98
Bag: 99
Bag: 100
Bag: 101
Bag: 102
Bag: 103
Bag: 104
Bag: 105
Bag: 106
Bag: 107
Bag: 108
Bag: 109
Bag: 110
Bag: 111
Bag: 112
Bag: 113
Bag: 114
Bag: 115
Bag: 116
Bag: 117
Bag: 118
Bag: 119
Bag: 120
Bag: 121
Bag: 122
Bag

This got 0.7482
Climbed from 132 --> 115 (17 place jump)